In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import math
import random
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import graph
from graph import LocMap, ConflictGraph, DistMatrixReconstruction
from collections import defaultdict
from labellines import labelLine, labelLines

from cvxpy import *

In [3]:
def matrixInverse(inputM):
            e0 = inputM[0, 0]
            e1 = inputM[0, 1]
            e2 = inputM[1, 0]
            e3 = inputM[1, 1]
            m = 1.0 / (e0*e3 - e1*e2)
            matrix = np.array([[m*e3, m*(-e1)], [m*(-e2), m*e0]])
            return matrix
        
matrix = np.array([[2, 1], [3, 2]])
matrixInverse(matrix)

array([[ 2., -1.],
       [-3.,  2.]])

In [4]:
def rmse(A,B):
    rmse = np.sqrt(np.mean(((A-B)**2)))
    return rmse

In [5]:
def construct_edges(node_num, p2p_graph, dist_matrix, num_edges):
    link_edges = defaultdict(dict)
    edge_no = 0
    
    labels = {}
    comm_edges_list = []

    for node_u in range(node_num):
        for node_v in range(node_u+1, node_num):
            if p2p_graph[node_u][node_v] == 1:
                link_edges[edge_no] = {'u':node_u, 'v':node_v, 'dist':dist_matrix[node_u][node_v]}
                edge_no += 1               

    assert (edge_no == num_edges)
    
    for edge_no in range(int(num_edges)):
        node_u = link_edges[edge_no]['u']
        node_v = link_edges[edge_no]['v']
        dist = link_edges[edge_no]['dist']
    
        comm_edges_list.append([node_u, node_v])
        labels[tuple([node_u, node_v])] = edge_no
    
    return link_edges, comm_edges_list, labels

def construct_conflict_edges(node_num, p2p_graph):
    
    conflict_edges_dict = defaultdict(dict)
    edge_no = 0

    conflict_edges_list = []

    for node_u in range(node_num):
        for node_v in range(node_u+1, node_num):
            if p2p_graph[node_u][node_v] == 1:
                conflict_edges_dict[edge_no] = {'u':node_u, 'v':node_v}
                edge_no += 1               
    
    for edge_no in range(len(conflict_edges_dict)):
        node_u = conflict_edges_dict[edge_no]['u']
        node_v = conflict_edges_dict[edge_no]['v']
    
        conflict_edges_list.append([node_u, node_v])
    
    return conflict_edges_dict, conflict_edges_list

In [ ]:
node_num = 30

nodes_pos = graph.generate_nodes_pos(node_num, -20, 20)

complete_dist_graph = graph.generate_complete_dist_graph(nodes_pos)

# nodes_pos = np.array([[0,0], [2,0], [0, -1], [6, -1], [6, 3], [4,4]])

In [ ]:
loc_pos = LocMap(nodes_pos, K=4)
loc_pos.HennenburgConstruction(random.randint(0, node_num))
loc_pos.ConstructEdgesK()
S = loc_pos.trilateration_ordering_check()


p2p_graph = loc_pos.get_graph()
dist_matrix = loc_pos.get_dist_matrix()

link_edges, comm_edges_list, labels = construct_edges(node_num, p2p_graph, dist_matrix, np.count_nonzero(p2p_graph == 1)/2)


In [7]:
nodes_pos = np.loadtxt("node_pos.csv", delimiter=",", dtype=np.double)
dist_matrix = np.loadtxt("dist_matrix.csv", delimiter=",", dtype=np.double)
estimated_dist_matrix = np.loadtxt("measured_dist_matrix.csv", delimiter=",", dtype=np.double)
active_EDM = np.loadtxt("active_EDM.csv", delimiter=",", dtype=np.double)
passive_EDM = np.loadtxt("passive_EDM.csv", delimiter=",", dtype=np.double)

p2p_graph = np.zeros((dist_matrix.shape[0], dist_matrix.shape[1]))
assert (p2p_graph.shape[0] == p2p_graph.shape[1] == 30)
    
for i in range(p2p_graph.shape[0]):
    for j in range(p2p_graph.shape[1]):
        p2p_graph[i, j] = 1 if estimated_dist_matrix[i, j] != 0 else 0
        
active_p2p_graph = np.zeros((dist_matrix.shape[0], dist_matrix.shape[1]))
assert (active_p2p_graph.shape[0] == active_p2p_graph.shape[1] == 30)
    
for i in range(active_p2p_graph.shape[0]):
    for j in range(active_p2p_graph.shape[1]):
        active_p2p_graph[i, j] = 1 if active_EDM[i, j] != 0 else 0
        
passive_p2p_graph = np.zeros((dist_matrix.shape[0], dist_matrix.shape[1]))
assert (passive_p2p_graph.shape[0] == passive_p2p_graph.shape[1] == 30)
    
for i in range(passive_p2p_graph.shape[0]):
    for j in range(passive_p2p_graph.shape[1]):
        passive_p2p_graph[i, j] = 1 if passive_EDM[i, j] != 0 else 0

In [39]:
passive_p2p_cpy = passive_p2p_graph.copy()

for i in range(passive_p2p_cpy.shape[0]):
    for j in range(i+1, passive_p2p_cpy.shape[1]):
        if passive_p2p_cpy[i, j] != 0 and i != j:
            if random.random() >= 2:
                passive_p2p_cpy[i, j] = 0
                passive_p2p_cpy[j, i] = 0

new_passive = np.multiply(passive_p2p_cpy, passive_EDM)
new_EDM = active_EDM.copy()

for i in range(30):
    for j in range(i+1, 30):
        if new_passive[i, j] != 0:
            if (new_EDM[i, j] != 0):
                new_EDM[i, j] = (new_EDM[i, j] + new_passive[i, j]) / 2
                new_EDM[j, i] = (new_EDM[j, i] + new_passive[j, i]) / 2
            else:
                new_EDM[i, j] = (new_EDM[i, j] + new_passive[i, j])
                new_EDM[j, i] = (new_EDM[j, i] + new_passive[j, i])
            
new_p2p = np.zeros((30, 30))

for i in range(30):
    for j in range(30):
        if (new_EDM[i, j] != 0):
            new_p2p[i, j] = 1
            
assert(np.allclose(new_p2p, new_p2p.T, rtol=10e-5, atol=10e-5))

In [40]:
print (np.allclose(new_EDM, new_EDM.T, rtol=10e-5, atol=10e-5))
print ((np.count_nonzero(new_EDM)+new_EDM.shape[0]) / (new_EDM.shape[0] * (new_EDM.shape[0]-1)))
print ((np.count_nonzero(active_EDM)+30) / (np.count_nonzero(new_EDM)+30))

True
0.7954022988505747
0.2861271676300578


In [41]:
rmse(new_EDM, np.multiply(new_p2p, dist_matrix))

1.3487562484783873

In [84]:
recon = DistMatrixReconstruction(new_EDM, new_EDM.shape[0])

D = recon.EDM_Completion(0.1, 0.03, 500)

rmse(D, dist_matrix)

1.9549279479109807

In [85]:
from sklearn.manifold import MDS

mds = MDS(n_components=2, max_iter=3000, eps=1e-9, dissimilarity="precomputed", normalized_stress='auto')

node_locations = mds.fit_transform(D)
e = graph.mds_relative_to_absolute_scale(node_locations, [0, 1, 2], nodes_pos[:3])

rmse(e, nodes_pos)

0.42057353155168004

In [44]:
# node_idx = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L',
#             'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X',
#             'Y', 'Z', 'AA', 'AB', 'AC', 'AD', 'AE', 'AF', 'AG', 'AG', 'AI',
#             'AJ', 'AK', 'AL', 'AM', 'AN', 'AO', 'AP', 'AQ', 'AR', 'AS',
#             'AT', 'AU', 'AV', 'AW', 'AX', 'AY', 'AZ', 'BA', 'BB', 'BC',
#             'BD', 'BE', 'BF', 'BG', 'BH', 'BI', 'BJ', 'BK', 'BL', 'BM',
#             'BN', 'BO', 'BP', 'BQ', 'BR', 'BS', 'BT', 'BU', 'BV', 'BW',
#             'BX', 'BY', 'BZ']

link_edges, comm_edges_list, labels = construct_edges(node_num, p2p_graph, dist_matrix, np.count_nonzero(p2p_graph == 1)/2)
node_idx = [x for x in range(node_num)]

G = nx.Graph()
G.add_edges_from(comm_edges_list)
pos = nodes_pos
# pos = {0: (0, 0), 1: (2, 0), 2: (0, -1), 3: (6, -1), 4: (6, 3), 5: (4, 4)} 
plt.figure(figsize=(8,6))

plt.xlabel('X axis (m)', fontsize=16)
plt.ylabel('Y axis (m)', fontsize=16)

nx.draw(
    G, pos, edge_color='red', width=1, linewidths=1,
    node_size=300, node_color='Yellow', alpha=0.9,
    labels={node: node_idx[node] for node in G.nodes()}
)

nx.draw_networkx_edge_labels(
    G, pos,
    edge_labels=labels,
    font_color='black'
)

plt.axis('on')
plt.show()

NameError: name 'node_num' is not defined

In [ ]:
noise = np.random.normal(0,2.5,(node_num, node_num))
noise_symm = (noise + noise.T)/2

np.allclose(noise_symm, noise_symm.T, rtol=10e-5, atol=10e-5)

In [ ]:
masked_dist_matrix = np.multiply(p2p_graph, dist_matrix)
noise_matrix = np.multiply(p2p_graph, noise_symm)


# estimated_dist_matrix = masked_dist_matrix + noise_matrix

In [ ]:
rmse(D, dist_matrix)

In [ ]:
e

In [ ]:
nodes_pos

In [ ]:
rmse(e, nodes_pos)

In [ ]:
re_cp = graph.generate_complete_dist_graph(node_locations)
sa_cp = graph.generate_complete_dist_graph(nodes_pos)
e_cp = graph.generate_complete_dist_graph(e)

In [ ]:
independent_sets = {0: [0, 17, 4, 40, 81, 97, 60],
                    1: [1, 17, 60, 95, 4, 120, 40, 82],
                    2: [2, 15, 32, 41, 60, 113, 4, 95], 
                    3: [3, 17, 4, 40, 81, 97, 60], 
                    4: [4, 97, 40, 82, 60, 41, 120, 1], 
                    5: [5, 97, 40, 82, 60, 41, 21, 57], 
                    6: [6, 97, 40, 82, 60, 41, 21, 57], 
                    7: [7, 4, 82, 40, 17, 97, 34], 
                    8: [8, 4, 60, 95, 16, 15, 32, 107], 
                    9: [9, 4, 97, 126, 31, 15, 20, 60], 
                    10: [10, 17, 60, 4, 95, 40, 81], 
                    11: [11, 17, 60, 95, 4, 120, 40, 82], 
                    12: [12, 16, 15, 32, 44, 95, 4], 
                    13: [13, 32, 41, 60, 97, 4, 107, 75], 
                    14:[14, 4, 82, 97, 60, 41, 120, 1], 
                    15: [15, 32, 4, 97, 107, 60, 41, 77, 27], 
                    16: [16, 15, 32, 60, 97, 4, 107, 0], 
                    17: [17, 40, 4, 82, 97, 60, 120, 1], 
                    18: [18, 40, 4, 82, 97, 60, 120, 11], 
                    19: [19, 60, 15, 32, 97, 4, 107, 0], 
                    20: [20, 15, 32, 60, 97, 4, 107, 8],
                    21: [21, 40, 41, 60, 82, 97, 4, 57],
                    22: [22, 40, 82, 41, 60, 97, 4, 75],
                    23: [23, 40, 4, 82, 97, 60, 120, 144], 
                    24: [24, 40, 4, 82, 97, 60, 57], 
                    25: [25, 40, 82, 41, 60, 97, 4, 75], 
                    26: [26, 40, 4, 82, 97, 60, 57], 
                    27: [27, 15, 32, 41, 60, 44, 95, 4], 
                    28:[28, 15, 32, 41, 60, 95, 113, 4], 
                    29: [29, 15, 32, 41, 60, 95, 113, 4], 
                    30: [30, 4, 97, 81, 60, 41, 77, 21], 
                    31: [31, 4, 97, 126, 15, 60, 41, 52, 76], 
                    32: [32, 4, 97, 15, 107, 60, 41, 77, 27], 
                    33: [33, 95, 15, 32, 44, 4], 
                    34: [34, 95, 4, 17, 40, 82, 7], 
                    35: [35, 97, 4, 17, 40, 82, 7], 
                    36: [36, 95, 4, 17, 40, 82, 7], 
                    37: [37, 82, 41, 60, 97, 4,75], 
                    38: [38, 4, 97, 107, 60, 41, 77, 21], 
                    39: [39, 4, 97, 126, 60, 41, 0], 
                    40: [40, 4, 82, 97, 60, 41, 120, 1], 
                    41: [41, 60, 97, 4, 40, 82, 120, 1], 
                    42: [42, 15, 32, 60, 97, 4, 107, 8], 
                    43: [43, 32, 4, 15, 97, 20, 60], 
                    44: [44, 32, 15, 4, 97, 77, 60, 27, 41], 
                    45: [45, 4, 120, 97, 60, 11, 16], 46: [46, 60, 15, 32, 44, 95, 4], 47: [47, 60, 15, 3
2, 95, 113, 4], 48: [48, 97, 15, 32, 4, 43], 49: [49, 60, 15, 32, 95, 44, 4], 50: [50, 15, 32, 95, 44, 4], 51: [51, 40, 82, 41, 60, 97, 4, 75], 52: [52, 15, 32, 41, 60, 4, 1
7, 97, 76], 53: [53, 15, 32, 41, 60, 95, 113, 4], 54: [54, 82, 40, 97, 60, 41, 1], 55: [55, 97, 40, 82, 60, 41, 85, 21], 56: [56, 97, 40, 82, 60, 41, 85, 21], 57: [57, 4, 8
2, 40, 97, 60, 21, 41], 58: [58, 4, 60, 95, 82, 40, 21, 41], 59: [59, 4, 82, 40, 17, 97, 34], 60: [60, 97, 4, 40, 82, 41, 120, 1], 61: [61, 97, 4, 41, 40, 82, 120, 2], 62: [
62, 97, 4, 40, 82, 120, 11, 16], 63: [63, 40, 4, 82, 97, 60, 120, 11], 64: [64, 40, 60, 97, 4, 82, 7], 65: [65, 32, 4, 15, 97, 20, 60], 66: [66, 4, 60, 95, 82, 40, 21, 41],
67: [67, 4, 97, 82, 40, 60, 21, 41], 68: [68, 40, 4, 97, 143, 21, 41], 69: [69, 4, 82, 40, 17, 97, 34], 70: [70, 31, 4, 15, 97, 20, 60], 71: [71, 4, 97, 15, 60, 41, 120, 1],
 72: [72, 4, 97, 126, 15, 60, 41, 52, 76], 73: [73, 4, 97, 15, 60, 41, 120, 1], 74: [74, 4, 97, 126, 60, 41, 0], 75: [75, 4, 97, 60, 41, 21, 40, 44], 76: [76, 4, 97, 60, 41,
 52, 15, 32, 107], 77: [77, 4, 97, 60, 107, 32, 15, 41, 27], 78: [78, 40, 82, 41, 60, 97, 4, 76], 79: [79, 40, 41, 60, 82, 95, 4, 120], 80: [80, 41, 60, 97, 4, 126, 75], 81:
 [81, 30, 4, 97, 60, 41, 77, 21], 82: [82, 40, 4, 97, 60, 41, 120, 1], 83: [83, 4, 95, 60, 41, 21, 40, 44], 84: [84, 95, 60, 4, 41, 21, 40, 81], 85: [85, 60, 97, 4, 41, 21,
40, 44], 86: [86, 40, 82, 60, 95, 4, 120], 87: [87, 4, 40, 81, 97, 34], 88: [88, 40, 60, 97, 4, 82, 7], 89: [89, 97, 40, 82, 60, 41, 21, 57], 90: [90, 97, 60, 107, 32, 15, 4
1, 27], 91: [91, 97, 40, 82, 60, 41, 21, 57], 92: [92, 4, 95, 60, 41, 21, 40, 81], 93: [93, 60, 4, 97, 41, 52, 15, 32, 107], 94: [94, 4, 95, 60, 41, 13, 32, 44], 95: [95, 60
, 4, 40, 82, 41, 120, 1], 96: [96, 60, 4, 40, 82, 120, 41, 1], 97: [97, 4, 40, 82, 60, 41, 120, 1], 98: [98, 97, 4, 40, 82, 120, 2], 99: [99, 97, 4, 40, 82, 120, 11, 17], 10
0: [100, 97, 4, 41, 40, 82, 120, 2], 101: [101, 97, 4, 40, 82, 120, 144, 17], 102: [102, 4, 40, 82, 120, 41, 1], 103: [103, 97, 4, 40, 82, 120, 11, 16], 104: [104, 95, 60, 4
0, 82, 41, 21, 57], 105: [105, 40, 82, 60, 41, 54, 1], 106: [106, 95, 60, 40, 82, 41, 21, 57], 107: [107, 32, 4, 97, 15, 60, 41, 77, 27], 108: [108, 40, 4, 97, 60, 41, 77, 2
1], 109: [109, 40, 4, 97, 60, 41, 77, 21], 110: [110, 4, 97, 60, 41, 77, 27], 111: [111, 32, 4, 97, 60, 41, 77, 21], 112: [112, 32, 4, 15, 60, 41, 77, 27], 113: [113, 32, 4,
 97, 15, 60, 41, 52, 76], 114: [114, 15, 32, 41, 60, 95, 113, 4], 115: [115, 15, 32, 41, 60, 95, 113, 4], 116: [116, 17, 40, 4, 81, 97, 34], 117: [117, 15, 32, 41, 60, 95, 1
13, 4], 118: [118, 15, 32, 41, 60, 95, 113, 4], 119: [119, 82, 40, 97, 60, 41, 1], 120: [120, 4, 82, 40, 97, 60, 41, 1], 121: [121, 4, 97, 82, 40, 60, 41, 21], 122: [122, 4,
 31, 97, 15, 60, 41, 1], 123: [123, 60, 95, 41, 21, 40, 81], 124: [124, 97, 60, 41, 27, 15, 32, 44], 125: [125, 97, 40, 82, 60, 41, 21, 57], 126: [126, 4, 31, 97, 15, 60, 41
, 52, 76], 127: [127, 41, 60, 15, 32, 113, 4, 95], 128: [128, 4, 95, 60, 41, 21, 40, 81], 129: [129, 17, 60, 4, 95, 40, 81], 130: [130, 4, 60, 95, 16, 15, 32, 107], 131: [13
1, 4, 97, 40, 82, 120, 143, 21], 132: [132, 60, 15, 32, 97, 4, 107, 8], 133: [133, 15, 32, 41, 60, 95, 113, 4], 134: [134, 15, 41, 60, 32, 113, 4, 95], 135: [135, 15, 32, 11
3, 4, 41, 60, 95], 136: [136, 41, 60, 15, 32, 113, 4, 95], 137: [137, 15, 32, 113, 4, 41, 60, 95], 138: [138, 15, 32, 60, 95, 113, 4], 139: [139, 4, 40, 82, 120, 41, 2], 140
: [140, 4, 60, 40, 82, 41, 120, 1], 141: [141, 60, 4, 40, 82, 120, 41, 1], 142: [142, 4, 60, 40, 82, 41, 120, 1], 143: [143, 97, 4, 120, 82, 40, 41, 21], 144: [144, 60, 41,
97, 4, 120, 13, 31], 145: [145, 60, 4, 120, 82, 40, 41, 21], 146: [146, 4, 40, 82, 120, 41, 2], 147: [147, 97, 4, 40, 82, 120, 2], 148: [148, 4, 95, 60, 41, 13, 32, 44], 149
: [149, 4, 97, 60, 16, 15, 32, 107], 150: [150, 97, 60, 107, 32, 15, 41, 27], 151: [151, 4, 60, 107, 32, 15, 41, 27], 152: [152, 4, 60, 107, 32, 15, 41, 27], 153: [153, 40,
4, 97, 60, 41, 120, 1], 154: [154, 4, 60, 40, 82, 120, 41, 1], 155: [155, 4, 40, 82, 60, 41, 120, 1], 156: [156, 17, 60, 4, 95, 40, 81], 157: [157, 60, 15, 32, 44, 95, 4], 1
58: [158, 4, 97, 60, 41, 120, 1], 159: [159, 4, 97, 60, 41, 120, 1], 160: [160, 40, 4, 97, 60, 41, 21, 75], 161: [161, 15, 32, 4, 97, 107, 8], 162: [162, 97, 4, 40, 82, 120,
 11, 17], 163: [163, 15, 32, 113, 4, 97, 60], 164: [164, 31, 15, 4, 20, 97, 60], 165: [165, 60, 40, 82, 41, 21, 57], 166: [166, 31, 97, 15, 60, 41, 52, 93], 167: [167, 32, 6
0, 97, 4, 107, 8], 168: [168, 15, 32, 97, 4, 107, 8], 169: [169, 4, 40, 82, 120, 11, 16], 170: [170, 4, 60, 40, 82, 120, 41, 1], 171: [171, 4, 40, 82, 120, 11, 16], 172: [17
2, 4, 97, 31, 15, 16, 0], 173: [173, 4, 31, 15, 60, 41, 52, 76]}

total_element_cnt = 0

for i in independent_sets:
    print (i)

In [ ]:
e

In [ ]:
nodes_pos

In [ ]:
node_locations

In [ ]:
rmse(e_cp, sa_cp)

In [ ]:
re_cp

In [ ]:
sa_cp

In [ ]:
def euclidean_dist(pos_a, pos_b):
    return math.dist(pos_a, pos_b)

def mds_relative_to_absolute_scale(estimated_coordinates, indices_of_anchors, anchors_true_coordinates):
    A = np.transpose(estimated_coordinates[indices_of_anchors,:])
    B = np.transpose(anchors_true_coordinates)
    estimated_coordinates = np.transpose(estimated_coordinates)

    assert (A.shape == B.shape)
    
    print ("A: \n{}".format(A))
    print ("B: \n{}".format(B))

    all_estimate_coordinates = estimated_coordinates
    num_of_points = estimated_coordinates.shape[1]
    
    print (num_of_points)
    
    def rigid_transform_3D(A,B):
        dimension, num_anchors = A.shape
        
        # find scale
        anchor_dist_A = np.zeros((num_anchors, num_anchors))
        anchor_dist_B = np.zeros((num_anchors, num_anchors))

        for i in range(num_anchors):
            for j in range(num_anchors):
                anchor_dist_A[i][j] = euclidean_dist(A[:, i], A[:, j])
                anchor_dist_B[i][j] = euclidean_dist(B[:, i], B[:, j])
        
        scale_matrix = np.divide(anchor_dist_B, anchor_dist_A, out=np.zeros_like(anchor_dist_A), where=anchor_dist_B!=0)
        total_n = 0
        cnt = 0
        for i in range(0, scale_matrix.shape[1]):
            for j in range(i+1, scale_matrix.shape[1]):
                total_n += scale_matrix[i, j]
                cnt += 1
        scale = total_n / cnt
        A = scale*A

        # find rotation
        centroid_A = np.expand_dims(np.mean(A, axis=1), axis=1)
        centroid_B = np.expand_dims(np.mean(B, axis=1), axis=1)
        
        diff_A = A - np.tile(centroid_A, (1, num_anchors))
        diff_B = B - np.tile(centroid_B, (1, num_anchors))

        H = diff_A @ np.transpose(diff_B)

        u, _, v = np.linalg.svd(H)

        rotation = v.T @ u.T

        # find translation
        translation = np.matmul(-rotation, centroid_A) + centroid_B

        return rotation, translation, scale

    rotation, translation, scale = rigid_transform_3D(A, B)
    
    # print ("Rotation: \n{},\nTranslation: \n{},\nScale: {}".format(rotation, translation, scale))

    estimated_coord_mds = np.matmul(rotation, (scale*all_estimate_coordinates)) + np.tile(translation, (1, num_of_points))

    return np.transpose(estimated_coord_mds)

In [ ]:
e = mds_relative_to_absolute_scale(node_locations, [0, 1, 2], nodes_pos[:3])


print ()
print (nodes_pos)
print (e)


In [ ]:
node_idx = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L',
            'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X',
            'Y', 'Z', 'AA', 'AB', 'AC', 'AD', 'AE', 'AF', 'AG', 'AG', 'AI',
            'AJ', 'AK', 'AL', 'AM', 'AN', 'AO', 'AP', 'AQ', 'AR', 'AS',
            'AT', 'AU', 'AV', 'AW', 'AX', 'AY', 'AZ', 'BA', 'BB', 'BC',
            'BD', 'BE', 'BF', 'BG', 'BH', 'BI', 'BJ', 'BK', 'BL', 'BM',
            'BN', 'BO', 'BP', 'BQ', 'BR', 'BS', 'BT', 'BU', 'BV', 'BW',
            'BX', 'BY', 'BZ']

G = nx.Graph()
G.add_edges_from(comm_edges_list)
pos = nodes_pos
# pos = {0: (0, 0), 1: (2, 0), 2: (0, -1), 3: (6, -1), 4: (6, 3), 5: (4, 4)} 
plt.figure(figsize=(8,6))

plt.xlabel('X axis (m)', fontsize=16)
plt.ylabel('Y axis (m)', fontsize=16)

nx.draw(
    G, pos, edge_color='red', width=1, linewidths=1,
    node_size=300, node_color='Yellow', alpha=0.9,
    labels={node: node_idx[node] for node in G.nodes()}
)

nx.draw_networkx_edge_labels(
    G, pos,
    edge_labels=labels,
    font_color='black'
)

plt.axis('on')
plt.show()

In [ ]:
node_idx = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L',
            'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X',
            'Y', 'Z', 'AA', 'AB', 'AC', 'AD', 'AE', 'AF', 'AG', 'AG', 'AI',
            'AJ', 'AK', 'AL', 'AM', 'AN', 'AO', 'AP', 'AQ', 'AR', 'AS',
            'AT', 'AU', 'AV', 'AW', 'AX', 'AY', 'AZ', 'BA', 'BB', 'BC',
            'BD', 'BE', 'BF', 'BG', 'BH', 'BI', 'BJ', 'BK', 'BL', 'BM',
            'BN', 'BO', 'BP', 'BQ', 'BR', 'BS', 'BT', 'BU', 'BV', 'BW',
            'BX', 'BY', 'BZ']

G = nx.Graph()
G.add_edges_from(comm_edges_list)
pos = e
# pos = {0: (0, 0), 1: (2, 0), 2: (0, -1), 3: (6, -1), 4: (6, 3), 5: (4, 4)} 
plt.figure(figsize=(8,6))

plt.xlabel('X axis (m)', fontsize=16)
plt.ylabel('Y axis (m)', fontsize=16)

nx.draw(
    G, pos, edge_color='red', width=1, linewidths=1,
    node_size=300, node_color='Yellow', alpha=0.9,
    labels={node: node_idx[node] for node in G.nodes()}
)

nx.draw_networkx_edge_labels(
    G, pos,
    edge_labels=labels,
    font_color='black'
)

plt.axis('on')
plt.show()

In [ ]:
conf_graph_obj = ConflictGraph(node_num, nodes_pos, dist_matrix, p2p_graph, link_edges)

conf_graph_obj.construct_conflict_graph()
conf_graph_obj.construct_independent_sets_by_edges()

conf_graph = conf_graph_obj.get_conflict_graph()
independent_sets = conf_graph_obj.get_independent_sets()

print (conf_graph)

_, conf_links = construct_conflict_edges(len(link_edges), conf_graph)

In [ ]:
print (conf_links)

In [ ]:
print (independent_sets)

In [ ]:
import matplotlib as mpl

G = nx.Graph()
G.add_edges_from(conf_links)
pos = nx.spring_layout(G)
plt.figure(figsize=(8,6))

cmap = plt.cm.plasma

nx.draw(
    G, pos, edge_color='black', width=2, linewidths=2,
    node_size=300, node_color='pink', alpha=0.9, arrowstyle="->",
    arrowsize=25,
    labels={node: node for node in G.nodes()}
)

# nodes = nx.draw_networkx_nodes(G, pos, node_size=300, node_color="indigo", alpha=)
# edges = nx.draw_networkx_edges(
#     G,
#     pos,
#     node_size=3,
#     arrowstyle="->",
#     arrowsize=10,
#     edge_color='black',
#     edge_cmap=cmap,
#     width=2,
# )
# set alpha value for each edge


ax = plt.gca()
ax.set_axis_off()
plt.show()

In [ ]:
relative_dist_graph = mds.matrix_mask(dist_matrix, p2p_graph)

# m = np.max(relative_dist_graph)
# relative_dist_graph /= m
# relative_dist_graph[relative_dist_graph == 0] = np.nan

In [ ]:
A = p2p_graph + np.eye(5)

In [ ]:
np.linalg.matrix_rank(relative_dist_graph)

In [ ]:
np.linalg.eigvals(relative_dist_graph)

In [ ]:
def rmse(A,B):
    rmse = np.sqrt(np.mean(((A-B)**2)))
    print("RMSE: %.2f" %rmse)
    return rmse

In [ ]:
from matrix_completion import svt_solve, nuclear_norm_solve, pmf_solve, biased_mf_solve

In [ ]:
from sklearn.impute import SimpleImputer
imp = SimpleImputer(missing_values=np.nan, strategy='mean')

a = imp.fit_transform(estimated_dist_matrix)

In [ ]:
rmse(dist_matrix, a)

In [ ]:
import cvxpy

X = Variable((30,30))
obj = Minimize(norm(X, 'nuc') )  # norm nuclear: hypotesis of low rank
constraints = [multiply(p2p_graph, X) == estimated_dist_matrix]
prob = Problem(obj, constraints)
prob.solve(solver=SCS)

In [ ]:
rmse(X.value, dist_matrix)